# SART Writer and Reader Tutorial
This tutorial will explain how to write SART (State, Action, Reward, Terminal) from episodes to hdf5 files, and read them back out.
## Sart Writer Tutorial
The SART Writer runs in a separate thread and takes SART tuples from a queue and writes them to hdf5 files.  One file per episode is written.  The files are saved with the following naming scheme:

             filepath/sart-name-episode-timestamp.hdf5
            
where name is a custom name given to tell what it is, episode is the episode number, and timestamp is the time in format ```'%Y%m%d-%H%M%S'``` at the creation of the file (not at close!).

In [1]:
import random, os, time
from collections import OrderedDict
from typing import Tuple, List, Union

import numpy as np

from psipy.rl.plant import Action, State, Plant
from psipy.rl.control.controller import Controller
from psipy.rl.control.controller import DiscreteRandomActionController
from psipy.rl.io.sart import SARTWriter, SARTReader


The writer takes a *directory* path, i.e., this directory will receive the file.  

The ```buffer_size``` parameter controls how big a resizable dataset inside the hdf5 file can get before increasing in size. This number shoud be tuned based on how big episodes usually are.  Too small and the writer will have to resize often; too big isn't much of a problem besides being inefficient.  At file close time, the excess "size" is trimmed off the datasets so that they only contain the data put into them (unused portions of the data are filled with np.nan).  

Lets generate a fake plant and other objects in order to demo the writer.

In [2]:
class TestState(State):
    _channels = ("sensor1", "sensor2")

class TestAction(Action):
    dtype = "discrete"
    num_values = 1
    channels = ("act1", "act1/2")  # / will transform to | (pipe) since it conflicts with hdf5 group paths!
    legal_values = (range(101), range(101))

class TestPlant(Plant):
    state_type = TestState
    action_type = TestAction

    def check_initial_state(self, state: State) -> State:
        return self._get_next_state(TestState({"sensor1": 10, "sensor2":20}), None)

    def _get_next_state(self, state: State, action: Action) -> State:
        state.terminal = False
        state.reward = 1
        return TestState({"sensor1": 10, "sensor2": 20})

    def notify_episode_stops(self) -> bool:
        pass

plant = TestPlant()
name = "TutorialRun"
logdir = os.path.join(".", "tutorial-sart-logs")
max_steps = 15  # just how many loops we will do in the "episode"

We now create the writer.  We set the buffer_size here to 5 so that we can see what happens when the buffer resizes.

In [3]:
sart_writer = SARTWriter(logdir, name, episode=1, buffer_size=5)

Now we simulate a loop during an episode.  At the end of each step, the state, action, and meta information is appended to the writer as a dict of dicts.

At the end of the episode, the writer is notified to close via ```.notify_episode_stops()```.

In [4]:
plant.notify_episode_starts()
state = plant.check_initial_state(None)

drc = DiscreteRandomActionController(TestState.channels(), TestAction)

for steps in range(1, max_steps + 1):
    action = drc.get_action(state)
    next_state = plant.get_next_state(state, action)
    reward = plant.get_cost(next_state)
    terminal = plant.is_terminal(next_state)

    sart_writer.append(
        {
            "state": state.as_dict(),
            "action": action.as_dict(),
            "meta": OrderedDict(meta=OrderedDict(world=43110)),
        }
    )

    state = next_state

plant.notify_episode_stops()
sart_writer.notify_episode_stops()

Now that the hdf5 file is written, let's quickly look inside manually to see the structure before we use the reader.

In [5]:
import h5py

filename = os.listdir("tutorial-sart-logs/")[0]
f = h5py.File(os.path.join("tutorial-sart-logs/", filename), 'r')
print(f.keys())
print(f['state'].keys())
print(f['state']['values'].keys())
print(f['state']['values']['sensor2'][:])
print(f['state']['values']['sensor2'][:].shape)
print(f['action']['act1'][:])

<KeysViewHDF5 ['action', 'state']>
<KeysViewHDF5 ['cost', 'terminal', 'values']>
<KeysViewHDF5 ['sensor1', 'sensor2']>
[20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
(15,)
[14. 62. 30. 58. 69. 75.  1. 70. 16. 23. 88.  6. 26. 55. 37.]


The order of the state, action, and meta channels are saved as bytes in the file's attributes, so that a ```state```/```action```/```meta``` object can be recreated when loading again.

In [6]:
print(f.attrs['state'][:])
print(f.attrs['action'][:])
print(f.attrs['meta'][:])
f.close()

['sensor1' 'sensor2']
['act1' 'act1/2']
[]


## Sart Reader Tutorial
The SART Reader loads a single episode file at a time.  Provide it simply the filepath to the hdf5 file.  Once created, it will read the file automatically.

In [7]:
reader = SARTReader(os.path.join('tutorial-sart-logs', filename))

Currently, only extracting the full episode is possible.  Full episode loading extracts the data into a tuple in the format expected by the ```Episode``` class, i.e. (observations, actions, terminals, costs).

In [8]:
reader.load_full_episode()

(array([[10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.],
        [10., 20.]], dtype=float32),
 array([[14., 50.],
        [62., 70.],
        [30., 17.],
        [58., 95.],
        [69., 82.],
        [75., 28.],
        [ 1., 31.],
        [70., 21.],
        [16., 30.],
        [23., 10.],
        [88., 54.],
        [ 6., 48.],
        [26., 57.],
        [55., 97.],
        [37., 68.]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32))

In [9]:
reader.close()

This can also be done for multiple files via the ```Batch``` class's classmethod, ```from_hdf5```.  Feel free to run the writing section multiple times to generate more hdf5 files to load.

In [10]:
from psipy.rl.io.batch import Batch

batch = Batch.from_hdf5(os.path.join('tutorial-sart-logs'), 
                        lookback=5)

print()
print(batch)
print(batch.nextstates[0])

/Users/slange/Code/sabbatical/dm-psiori/psipy/psipy/rl/io/batch.py:71: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  self._terminals = np.asarray(terminals, dtype=np.bool).ravel()


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

### SWMR
SART Writer versions 2.0.0 and greater have "single write, multiple read" (SWMR) mode activated.  That means that the SART Reader can read a SART file while it is being written.  This is useful for dashboards and the like which want to display data.

### Run the below cell to remove the sart data written during the tutorial.

In [ ]:
import shutil
shutil.rmtree('tutorial-sart-logs')